# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [1]:
sc

Calculation started (calculation_id=78c5be78-8e0f-b0d6-4613-50f08ae44705) in (session=4cc5be77-af71-c190-344d-5fa34856b792). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
<SparkContext master=athena appName=default>



<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [2]:
#export
from pyspark.sql.functions import *
from pyspark.sql import *

Calculation started (calculation_id=f8c5be78-941e-88ef-2cb4-7dc3bda2b4a9) in (session=4cc5be77-af71-c190-344d-5fa34856b792). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [3]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

Calculation started (calculation_id=74c5be78-9955-9d42-686b-6b47afc36a11) in (session=4cc5be77-af71-c190-344d-5fa34856b792). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [4]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'jag31'

Calculation started (calculation_id=32c5be78-a65d-4cd3-7a3e-fe5c871bd438) in (session=4cc5be77-af71-c190-344d-5fa34856b792). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [5]:
#export
from pyspark.sql.functions import col
def long_trips(trips):
    # Returns a Dataframe (trips) with Schema the same as :trips:
    long_trips = trips.filter(col("trip_distance") >= 2.0)
    return long_trips

Calculation started (calculation_id=8cc5be78-b107-bb53-c45f-54b83ca266f0) in (session=4cc5be77-af71-c190-344d-5fa34856b792). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [6]:
#export
from pyspark.sql.functions import sum
def manhattan_trips(trips, lookup):
    # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount
    
    # Join 'trips' and 'lookup' DataFrames on 'DOLocationID' and 'LocationID'
    joined_df = trips.join(lookup, trips['DOLocationID'] == lookup['LocationID'], 'inner')

    # Filter the joined DataFrame to keep only those with drop-off location in Manhattan and trip_distance >= 2.0
    manhattan_trips = joined_df.filter((joined_df['Borough'] == 'Manhattan') & (joined_df['trip_distance'] >= 2.0))

    # Group by DOLocationID and calculate the sum of passenger_count for each location
    grouped = manhattan_trips.groupBy('DOLocationID').agg(sum('passenger_count').alias('pcount'))

    # Sort the DataFrame in descending order of pcount
    sorted_mtrips = grouped.sort('pcount', ascending=False)

    # Select the top 20 locations
    top_20_mtrips = sorted_mtrips.limit(20)

    return top_20_mtrips

Calculation started (calculation_id=ecc5be78-bfa3-1f0f-a493-5651442e6f06) in (session=4cc5be77-af71-c190-344d-5fa34856b792). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [7]:
#export
from pyspark.sql.functions import avg, count
from pyspark.sql.types import DecimalType
from pyspark.sql.functions import col

def weighted_profit(trips, mtrips):
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    mtrips_copy = mtrips
    mtrips_copy = mtrips_copy.withColumnRenamed('DOLocationID', "DOLocationID_Top")
    trips_filtered = trips.join(mtrips_copy, trips["DOLocationID"] == mtrips_copy["DOLocationID_Top"], "inner").select("PULocationID", "DOLocationID","total_amount")
    trips_filtered_count = trips_filtered.groupBy(["PULocationID"]).count()
    trips_count = trips.groupBy(["PULocationID"]).agg({"*":"count", "total_amount":"mean"})
    trips_filtered_count = trips_filtered_count.withColumnRenamed('PULocationID', "PULocationID_Top")
    trips_count = trips_count.withColumnRenamed('avg(total_amount)', "avg_per_trip")
    trips_count = trips_count.withColumnRenamed('count(1)', "count_total")
    result_df = trips_count.join(trips_filtered_count, trips_count["PULocationID"] == trips_filtered_count["PULocationID_Top"], "inner").select("PULocationID","count","count_total","avg_per_trip")
    #return trips_count.orderBy(["PULocationID"]), trips_filtered_count.orderBy(["PULocationID"]), result_df.orderBy(["PULocationID"])
    result_df = result_df.withColumn("avg_per_trip", result_df["avg_per_trip"].cast("decimal(20,10)"))
    result_df = result_df.withColumn("count", result_df["count"].cast("decimal(20,10)"))
    result_df = result_df.withColumn("count_total", result_df["count_total"].cast("decimal"))
    result_df = result_df.withColumn("weighted_profit",(result_df["avg_per_trip"]*result_df["count"])/result_df["count_total"])
    return result_df.orderBy(["PULocationID"]).select(["PULocationID", "weighted_profit"])

Calculation started (calculation_id=74c5be79-5047-08da-ac5b-35ec1ad2f2ac) in (session=4cc5be77-af71-c190-344d-5fa34856b792). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [8]:
#export 
def final_output(wp, lookup): 
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    final = wp.join(lookup, wp.PULocationID == lookup.LocationID, 'inner')
    final = final.select(
        col("Zone").alias("Zone"),
        col("Borough").alias("Borough"),
        col("weighted_profit").cast(DecimalType(38, 8))  # Maintain consistency with wp
    )
    
    final = final.orderBy(col("weighted_profit").desc()).limit(20)

    return final

Calculation started (calculation_id=6ec5be79-5fcf-7130-fa0b-4dc78197fca5) in (session=4cc5be77-af71-c190-344d-5fa34856b792). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [9]:
# Sample data for testing
size = 'large'  # You can change this to 'large' if needed
bucket = "s3://cse6242-jag31/output-large9"  # Replace with your S3 bucket name

# Call the main() function with 'small' data
main(size, bucket)

Calculation started (calculation_id=8ec5be79-7f11-debf-3c28-127500a50c9d) in (session=4cc5be77-af71-c190-344d-5fa34856b792). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
jag31
Trip Count:  187203269
+--------------------+-------------+---------------+
|                Zone|      Borough|weighted_profit|
+--------------------+-------------+---------------+
|        Baisley Park|       Queens|    29.36045578|
|Flushing Meadows-...|       Queens|    27.30484573|
|       South Jamaica|       Queens|    26.29491624|
|     Randalls Island|    Manhattan|    24.15098994|
|        Astoria Park|       Queens|    21.70641711|
|Briarwood/Jamaica...|       Queens|    19.94506463|
|Springfield Garde...|       Queens|    19.46830929|
|             Jamaica|       Queens|    19.28394300|
|              Corona|       Queens|    18.22876925|
|   LaGuardia Airport|       Queens|    18.18133881|
|         Jamaica Bay|       Queens|    17.10052945|
|             Maspeth|       Queens|    17.00545064|
|Eltingville/Annad...|Staten Island|    16.83776476|
|         JFK Airport|       Queens|    16.77772535|
|        Battery Park|    Manhattan|    12.8497

In [21]:
#trips, lookup = load_data(size='small')
#trips = long_trips(trips)
#mtrips = manhattan_trips(trips, lookup)

In [22]:
#wp = weighted_profit(trips, mtrips)
#final = final_output(wp, lookup)

In [23]:
#final.show(5)

In [24]:
#wp.show()